# DSAIT4335 Recommender Systems
# Final Project

In this project, you will work to build different recommendation models and evaluate the effectiveness of these models through offline experiments. The dataset used for the experiments is **MovieLens100K**, a movie recommendation dataset collected by GroupLens: https://grouplens.org/datasets/movielens/100k/. For more details, check the project description on Brightspace.

# Instruction

The MovieLens100K is already splitted into 80% training and 20% test sets. Along with training and test sets, movies metadata as content information is also provided.

**Expected file structure** for this assignment:   
   
   ```
   RecSysProject/
   ├── training.txt
   ├── test.txt
   ├── movies.txt
   └── codes.ipynb
   ```

**Note:** Be sure to run all cells in each section sequentially, so that intermediate variables and packages are properly carried over to subsequent cells.

**Note** Be sure to run all cells such that the submitted file contains the output of each cell.

**Note** Feel free to add cells if you need more for answering a question.

**Submission:** Answer all the questions in this jupyter-notebook file. Submit this jupyter-notebook file (your answers included) to Brightspace. Change the name of this jupyter-notebook file to your group number: example, group10 -> 10.ipynb.

# Setup

In [5]:
%pip install transformers torch  # For BERT
%pip install -r requirements.txt
# you can refer https://huggingface.co/docs/transformers/en/model_doc/bert for various versions of the pre-trained model BERT

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.sparse import csr_matrix
from scipy.spatial.distance import cosine, correlation
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from sklearn.preprocessing import StandardScaler, MultiLabelBinarizer
from transformers import logging 
from recommendation_algorithms.hybrid_recommender import HybridRecommender
from recommendation_algorithms.matrix_factorization import MatrixFactorizationSGD
from recommendation_algorithms.bayesian_probabilistic_ranking import BayesianProbabilisticRanking
from recommendation_algorithms.item_knn import ItemKNN
from recommendation_algorithms.user_knn import UserKNN
from recommendation_algorithms.content_based import ContentBasedRecommender
from evaluation.grid_search import grid_search
from evaluation.score_prediction_metrics import MAE, MSE, RMSE 
logging.set_verbosity_error()
import re
import time, math
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(10)

print("Libraries imported successfully!")

Libraries imported successfully!


# Load dataset

In [7]:
# loading the training set and test set
columns_name=['user_id','item_id','rating','timestamp']
train_data = pd.read_csv('data/training.txt', sep='\t', names=columns_name)
test_data = pd.read_csv('data/test.txt', sep='\t', names=columns_name)

display(train_data[['user_id','item_id','rating']].head())
print(f'The shape of the training data: {train_data.shape}')
print(f'The shape of the test data: {test_data.shape}')

movies = pd.read_csv('data/movies.txt',names=['item_id','title','genres','description'],sep='\t')
display(movies.head())

,user_id,item_id,rating
0,1,1,5
1,1,2,3
2,1,3,4
3,1,4,3
4,1,5,3


The shape of the training data: (80000, 4)
The shape of the test data: (20000, 4)


,item_id,title,genres,description
0,1,Toy Story (1995),"Animation, Children's, Comedy","A group of sentient toys, who pretend to be li..."
1,2,GoldenEye (1995),"Action, Adventure, Thriller","In 1986, MI6 agents James Bond and Alec Trevel..."
2,3,Four Rooms (1995),Thriller,"On New Year's Eve, bellhop Sam (Marc Lawrence)..."
3,4,Get Shorty (1995),"Action, Comedy, Drama",Chili Palmer is a Miami-based loan shark and m...
4,5,Copycat (1995),"Crime, Drama, Thriller",After giving a guest lecture on criminal psych...


# Task 1) Implementation of different recommendation models as well as a hybrid model combining those recommendation models

### Content-Based

<div style="background-color: #19e0d0ff; color:#FFFFFF"> 
Explanation for why our model works.
</div>


In [ ]:
BERT_MODEL_NAME = 'boltuix/bert-mini'
percentage = 0.005
movies_small = movies.iloc[0: int(percentage * len(movies))]
train_data_small = train_data[train_data["item_id"].isin(movies_small["item_id"])]
content = movies_small["title"] + movies_small["description"] + movies_small["description"]
CBR = ContentBasedRecommender(BERT_MODEL_NAME, train_data_small, 16, "weighted_average", content)
CBR.train(train_data_small)

100%|██████████| 6/6 [00:02<00:00,  2.85it/s]


#### Hyperparameter Tuning

In [ ]:
hyperparameters_content_based = {
    "aggregation_method": ["average", "weighted_average", "avg_pos"],
    "bert_model": ['boltuix/bert-mini', 'distilbert-base-uncased'],
    "data": [train_data_small],
    "batch_size": [16],
    "content": [content]
}

best_parameters_cb, params_cb = grid_search(hyperparameters_content_based, ContentBasedRecommender, train_data_small, RMSE)

 17%|█▋        | 1/6 [00:06<00:30,  6.04s/it]

Parameters with metric: 1.288531697168926


 33%|███▎      | 2/6 [00:15<00:33,  8.28s/it]

Parameters with metric: 1.2335756589178988


 50%|█████     | 3/6 [00:21<00:21,  7.04s/it]

...

Parameters with metric: 1.1981533919116554


 83%|████████▎ | 5/6 [00:37<00:07,  7.39s/it]

Parameters with metric: 0.8809316718676808


100%|██████████| 6/6 [00:45<00:00,  7.65s/it]

Parameters with metric: 0.8933349569478888
-----------------------------------
Best params metric 0.8809316718676808
Best params: [('aggregation_method', 'avg_pos'), ('bert_model', 'boltuix/bert-mini'), ('data',        user_id  item_id  rating  timestamp
0            1        1       5  874965758
1            1        2       3  876893171
2            1        3       4  878542960
3            1        4       3  876893119
4            1        5       3  889751712
...        ...      ...     ...        ...
79625      940        7       4  885921597
79626      940        8       5  885921577
79731      941        1       5  875049144
79732      941        7       4  875048952
79832      943        2       5  888639953

[1303 rows x 4 columns]), ('batch_size', 16), ('content', 0    Toy Story (1995)A group of sentient toys, who ...
1    GoldenEye (1995)In 1986, MI6 agents James Bond...
2    Four Rooms (1995)On New Year's Eve, bellhop Sa...
3    Get Shorty (1995)Chili Palmer is a Miami-ba

#### User-based Collaborative Filtering

In [10]:
u_knn = UserKNN(2)
u_knn.train(train_data_small)
u_knn.predict_score(4, 3)

586it [04:29,  2.18it/s]


0.0

In [11]:
hyperparameters_user_knn = {
    "k": [2, 3, 5, 7, 9, 11]
}


similarity_matrix = u_knn.similarity_matrix
best_parameters_cb, params_cb = grid_search(hyperparameters_user_knn, UserKNN, train_data_small, RMSE, similarity_matrix=similarity_matrix)

 17%|█▋        | 1/6 [00:05<00:27,  5.49s/it]

Parameters with metric: 1.5199708104952574


 33%|███▎      | 2/6 [00:10<00:21,  5.27s/it]

Parameters with metric: 1.525483256312132


 50%|█████     | 3/6 [00:15<00:15,  5.22s/it]

...

Parameters with metric: 1.5308169986539124


 83%|████████▎ | 5/6 [00:27<00:05,  5.66s/it]

Parameters with metric: 1.5297976698193505


100%|██████████| 6/6 [00:33<00:00,  5.66s/it]

Parameters with metric: 1.5261312523767967
-----------------------------------
Best params metric 1.5199708104952574
Best params: [('k', 2)]


### Item-based Collaborative Filtering

In [12]:
i_knn = ItemKNN(2)
i_knn.train(train_data_small)

8it [00:00, 122.36it/s]


#### Hyperparameter Tuning

In [13]:
percentage = 0.005
movies_small = movies.iloc[0: int(percentage * len(movies))]
train_data_small = train_data[train_data["item_id"].isin(movies_small["item_id"])]
content = movies_small["title"] + movies_small["description"] + movies_small["description"]

hyperparameters_item_knn = {
    "k": [2, 3, 5, 7, 9, 11]
}

similarity_matrix = i_knn.similarity_matrix
best_parameters_cb, params_cb = grid_search(hyperparameters_item_knn, ItemKNN, train_data_small, RMSE, similarity_matrix=similarity_matrix)

 17%|█▋        | 1/6 [00:07<00:35,  7.01s/it]

Parameters with metric: 0.7173769554537288


 33%|███▎      | 2/6 [00:12<00:24,  6.05s/it]

Parameters with metric: 0.7151230434899972


 50%|█████     | 3/6 [00:18<00:17,  5.93s/it]

...

Parameters with metric: 0.7147774615106942


 83%|████████▎ | 5/6 [00:30<00:06,  6.12s/it]

Parameters with metric: 0.7147774615106942


100%|██████████| 6/6 [00:36<00:00,  6.14s/it]

Parameters with metric: 0.7147774615106942
-----------------------------------
Best params metric 0.7147774615106942
Best params: [('k', 7)]


### Matrix Factorization

In [14]:
matrix_factorization = MatrixFactorizationSGD()
bpr = BayesianProbabilisticRanking()
rating_recommenders = [matrix_factorization]
ranking_recommenders = [matrix_factorization, bpr]
max_k = 10 # Recommendation list size
hybrid_recommender = HybridRecommender(train_data, rating_recommenders, ranking_recommenders, max_k, True)

Started training hybrid recommender on 943 users and 1650 items...
Training individual models...


100%|██████████| 5/5 [00:12<00:00,  2.54s/it]


Finished training individual models.
Started linear regression...
[INFO] Visualization saved to: plots/linear_regression_rating_2025-10-26_14-24-01.png
Finished rating linear regression, weights are:
  Matrix Factorization: 1.0054005607103984
Finished ranking linear regression, weights are:
  Matrix Factorization: 0.5
  Bayesian Probabilistic Ranking: 0.5
Precomputing predictions...
Finished computing predictions, model is ready to use.


In [15]:
user_id = 1
item_id = 2
predicted_score = hybrid_recommender.predict_score(user_id, item_id)
actual_score = train_data.loc[((train_data['user_id'] == user_id) & (train_data['item_id'] == item_id)), 'rating'].values[0]
print(f'Predicted score {predicted_score} for user {user_id} and item {item_id}, actual score: {actual_score}.')

predicted_ranking = hybrid_recommender.predict_ranking(user_id, max_k)
print(f"Predicted ranking for user {user_id}")
for i in range(len(predicted_ranking)):
    print(f"  {i + 1}: {predicted_ranking[i]}")

Predicted score 3.0590974553029033 for user 1 and item 2, actual score: 3.
Predicted ranking for user 1
  1: (np.float64(276.0), np.float64(2.544232296045881))
  2: (np.float64(100.0), np.float64(2.5241349110594973))
  3: (np.float64(318.0), np.float64(2.4717436182296435))
  4: (np.float64(483.0), np.float64(2.4449129128822844))
  5: (np.float64(50.0), np.float64(2.4369152190698156))
  6: (np.float64(191.0), np.float64(2.4356222128228513))
  7: (np.float64(151.0), np.float64(2.419328806851773))
  8: (np.float64(174.0), np.float64(2.413345689642898))
  9: (np.float64(357.0), np.float64(2.407086181586917))
  10: (np.float64(172.0), np.float64(2.3924889033024628))


#### Test BPR Out 

In [ ]:
from recommendation_algorithms.bayesian_probabilistic_ranking import BayesianProbabilisticRanking

bpr = BayesianProbabilisticRanking()
bpr.train(train_data)   

np.float64(3.954941986653667)

: 

: 

: 

## Hybrid Recommender

# Task 2) Experiments for both rating prediction and ranking tasks, and conducting offline evaluation

: 

: 

: 

# Task 3) Implement baselines for both rating prediction and ranking tasks, and perform experiments with those baselines

: 

: 

: 

# Task 4) Analysis of recommendation models. Analyzing the coefficients of hybrid model and the success of recommendation models for different users' groups. 

: 

: 

: 

# Task 5) Evaluation of beyond accuracy

: 

: 

: 